In [116]:
import pandas as pd
import numpy as np

In [122]:
from cobra.io import read_sbml_model, load_json_model, load_yaml_model, load_matlab_model, load_model
from cobra.test import create_test_model
from cobra.flux_analysis import flux_variability_analysis, pfba

## Reading the model
##### The model that describes your organism can be in different format. Uncomment one of the following command to read your file. Alternatively you can read the model from BIGG or BioModels database by knowing the id
##### Recomendations for the most popular models : 

In [61]:
# model = load_model("iML1515") ## iML1515 is the model ID

#path = 'path_to_model_file'

# read sbml model
# model = read_sbml_model(path)

# read json model
# model = load_json_model(path)

# read yml model
# model = load_yaml_model(path)

# read matlab model
# model = load_matlab_model(path)

In [92]:
# E. coli core metabolism model

model = create_test_model("textbook")

# Measured uptake rates and fluxes

##### The flux data should be in the format metabolite id or reaction ID, flux value, error/ci/std
##### Usually the units that are used are mmol/h/gDW

In [93]:
long2019 = pd.read_csv('long2019_tidy.csv')

In [94]:
long2019.head()

,Strain,Genotype,Medium,Dilution,Measurement_Type,Time,Measurement_ID,Original_ID,Value,Original_Value,Unit,Original_Unit,Error,Original_Error,Source
0,BW25113,WT,GLC,NaN,consumption_rate,NaN,qGlc,qGLC,8.58,8.58,mmol/h/g,mmol/h/gDW,NaN,NaN,"(Long, 2019)"
1,BW25113,glk,GLC,NaN,consumption_rate,NaN,qGlc,qGLC,8.05,8.05,mmol/h/g,mmol/h/gDW,NaN,NaN,"(Long, 2019)"
2,BW25113,pgm,GLC,NaN,consumption_rate,NaN,qGlc,qGLC,9.34,9.34,mmol/h/g,mmol/h/gDW,NaN,NaN,"(Long, 2019)"
3,BW25113,pgi,GLC,NaN,consumption_rate,NaN,qGlc,qGLC,2.06,2.06,mmol/h/g,mmol/h/gDW,NaN,NaN,"(Long, 2019)"
4,BW25113,pfkA,GLC,NaN,consumption_rate,NaN,qGlc,qGLC,2.46,2.46,mmol/h/g,mmol/h/gDW,NaN,NaN,"(Long, 2019)"


In [95]:
# Subset the data only for WT
data = long2019.query('Genotype == "WT"')

In [96]:
data.query('Measurement_ID == "qGlc"')

,Strain,Genotype,Medium,Dilution,Measurement_Type,Time,Measurement_ID,Original_ID,Value,Original_Value,Unit,Original_Unit,Error,Original_Error,Source
0,BW25113,WT,GLC,NaN,consumption_rate,NaN,qGlc,qGLC,8.58,8.58,mmol/h/g,mmol/h/gDW,NaN,NaN,"(Long, 2019)"


In [97]:
rxns = data['Measurement_ID'].unique()

In [98]:
# If there are reactions that are names differently than in the model, add them to this dictionary following the example
id_conc = {'qGlc':'EX_glc__D_e'}

In [ ]:
# You may need the conversion of the fluxes from metabolite names. For example, this will convert glc to EX_glc_e
# rates_ids = ['EX_' + met + '_e' for met in data['Metabolite ID'].values]
# data['Measurement_ID'] = rates_ids

## This code will go through dataframe and set boundaries for found reactions. 
#### Make sure the column names are the same as in the example Measurement_ID,Value,Error

In [99]:
added_rxns = [] 
for _, rxn in data.head().iterrows():
    try:
        model.reactions.get_by_id(rxn['Measurement_ID']).lower_bound = rxn['Value'] - rxn['Value']*0.025 # if there is no error/ci/std it is possible to use 5% interval
        model.reactions.get_by_id(rxn['Measurement_ID']).upper_bound = rxn['Value'] + rxn['Value']*0.025 # use rxn['Error'] instead of rxn['Value']*0.025 if error is specified
        added_rxns.append(rxn['Measurement_ID'])
    except KeyError:
        print(f"The reaction ID {rxn['Measurement_ID']} was not found in the model.")
        try:
            model.reactions.get_by_id(id_conc[rxn['Measurement_ID']]).lower_bound = rxn['Value'] - rxn['Value']*0.025 # if there is no error/ci/std it is possible to use 5% interval
            model.reactions.get_by_id(id_conc[rxn['Measurement_ID']]).upper_bound = rxn['Value'] + rxn['Value']*0.025
            print(f"The reaction ID {rxn['Measurement_ID']} was found in the dictionary.")
            added_rxns.append(id_conc[rxn['Measurement_ID']])
        except KeyError:
            print(f"The reaction ID {rxn['Measurement_ID']} was not found in the dictionary.")

The reaction ID qGlc was not found in the model.
The reaction ID qGlc was found in the dictionary.
The reaction ID GLCptspp was not found in the model.
The reaction ID GLCptspp was not found in the dictionary.


### Alternatively use this code to manually add reaction boundaries one by one

In [100]:
# reaction_id = 'ID' # add the reaction ID here
# model.reactions.get_by_id(reaction_id).lower_bound = -1000 # add measured value
# model.reactions.get_by_id(reaction_id).upper_bound = 1000 # add measured value

In [101]:
if model.reactions.get_by_id('EX_glc__D_e').lower_bound > 0:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -model.reactions.get_by_id('EX_glc__D_e').lower_bound
    model.reactions.get_by_id('EX_glc__D_e').upper_bound = -model.reactions.get_by_id('EX_glc__D_e').upper_bound

In [102]:
# list of the constrained reactions
added_rxns

['EX_glc__D_e', 'PGI', 'PFK', 'FBP']

# Biomass function

In [135]:
# to change the objective function
# model.objective = "Reaction ID"
# model.reactions.get_by_id("Reaction ID").upper_bound = 1000

# Run FBA

In [131]:
solution_FBA = model.optimize()
solution_FBA

,fluxes,reduced_costs
ACALD,0.000000,-4.336809e-19
ACALDt,0.000000,-0.000000e+00
ACKr,0.000000,1.734723e-18
ACONTa,6.021834,0.000000e+00
ACONTb,6.021834,0.000000e+00
...,...,...
TALA,0.815960,0.000000e+00
THD2,1.978888,0.000000e+00
TKT1,0.815960,0.000000e+00
TKT2,0.544520,0.000000e+00


In [132]:
FBA_fluxes = pd.DataFrame(solution_FBA.fluxes)
# FBA_fluxes.to_clipboard() # to copypaste
# FBA_fluxes.to_excel('name_of_the_excel.xlxs') # to save to excel
# FBA_fluxes.to_csv('name_of_the_excel.csv') # to save to csv

In [134]:
#  optimized objective value (usually biomass)
solution_FBA.objective_value

0.7519114393573396

# Run FVA

In [127]:
solution_FVA = flux_variability_analysis(model,fraction_of_optimum=0.9,loopless=True) # loopless=True ensured no loop in the solution, but may take more time to run
solution_FVA

,minimum,maximum
ACALD,-2.197078,0.000000
ACALDt,-2.197078,0.000000
ACKr,-3.237799,0.000000
ACONTa,3.840566,7.117883
ACONTb,3.840566,7.117883
...,...,...
TALA,0.717154,0.834550
THD2,0.000000,0.000000
TKT1,0.717154,0.834550
TKT2,0.445932,0.590254


In [128]:
# FVA_fluxes = pd.DataFrame(solution_FVA)
# FVA_fluxes.to_clipboard() # to copypaste
# FVA_fluxes.to_excel('name_of_the_excel.xlxs') # to save to excel
# FVA_fluxes.to_csv('name_of_the_excel.csv') # to save to csv

# Run pFBA

In [129]:
solution_pFBA = pfba(model)
solution_pFBA

,fluxes,reduced_costs
ACALD,0.000000,-2.0
ACALDt,0.000000,2.0
ACKr,0.000000,2.0
ACONTa,6.021834,-2.0
ACONTb,6.021834,-2.0
...,...,...
TALA,0.815960,-2.0
THD2,1.978888,-2.0
TKT1,0.815960,-2.0
TKT2,0.544520,-2.0


In [130]:
pFBA_fluxes = pd.DataFrame(solution_FBA.fluxes)
# pFBA_fluxes.to_clipboard() # to copypaste
# pFBA_fluxes.to_excel('name_of_the_excel.xlxs') # to save to excel
# pFBA_fluxes.to_csv('name_of_the_excel.csv') # to save to csv